In [20]:
# ComicCrafter AI - Fixed Implementation for Google Colab
# This version resolves package conflicts and includes error handling
!pip install gradio

import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import json
import gradio as gr
import traceback

# Install required packages without version constraints
!pip install -q diffusers transformers accelerate safetensors
!pip install -q opencv-python
# Install xformers - adding this explicit installation step
!pip install -q xformers

# Wait a moment for installations to complete
import time
time.sleep(3)

import cv2
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Enable debug logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU status
!nvidia-smi

# 1. SETUP LIGHTWEIGHT TEXT GENERATION MODEL
def setup_text_model():
    logger.info("Setting up story generation model...")
    try:
        # Load a smaller model for text generation
        model_name = "distilgpt2"  # Even smaller than GPT-2
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        logger.info(f"Successfully loaded {model_name}")
        return tokenizer, model
    except Exception as e:
        error_msg = f"Error setting up text model: {str(e)}"
        logger.error(error_msg)
        traceback.print_exc()
        raise Exception(error_msg)

# 2. SETUP LIGHTWEIGHT IMAGE GENERATION MODEL
def setup_image_model():
    logger.info("Setting up image generation model...")
    try:
        # Check if CUDA is available
        if not torch.cuda.is_available():
            logger.warning("CUDA not available! Falling back to CPU (this will be slow)")

        # Use an even smaller/faster model
        model_id = "CompVis/stable-diffusion-v1-4"  # Smaller than SD 2.0

        # Use efficient scheduler
        scheduler = DPMSolverMultistepScheduler.from_pretrained(
            model_id,
            subfolder="scheduler"
        )

        # Load the pipeline with low precision for memory efficiency
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            scheduler=scheduler,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            safety_checker=None,  # Disable for speed
            requires_safety_checker=False
        )

        # Move to appropriate device
        device = "cuda" if torch.cuda.is_available() else "cpu"
        pipe = pipe.to(device)

        # Enable memory saving features if on GPU
        if device == "cuda":
            # Try to enable xformers memory efficient attention if available
            try:
                import xformers
                if hasattr(pipe, "enable_xformers_memory_efficient_attention"):
                    pipe.enable_xformers_memory_efficient_attention()
                    logger.info("Successfully enabled xformers memory efficient attention")
            except ImportError:
                logger.warning("xformers not installed, falling back to attention slicing")

            # Enable attention slicing for lower memory usage regardless
            pipe.enable_attention_slicing(1)

        logger.info(f"Successfully loaded Stable Diffusion on {device}")
        return pipe
    except Exception as e:
        error_msg = f"Error setting up image model: {str(e)}"
        logger.error(error_msg)
        traceback.print_exc()
        raise Exception(error_msg)

# 3. STORY GENERATION FUNCTION
def generate_story(prompt, num_panels=4, tokenizer=None, model=None):
    logger.info(f"Generating story from prompt: {prompt}")

    try:
        # Create a simple story structure directly without using the model
        # This is more reliable than trying to get proper JSON from GPT-2
        story_data = {}

        # Simple story progression
        scenarios = [
            "introduction scene showing characters",
            "conflict or problem emerges",
            "characters dealing with the situation",
            "resolution or conclusion"
        ]

        # Ensure we have enough scenarios for requested panels
        while len(scenarios) < num_panels:
            scenarios.append("continued action")

        # Create panels based on prompt
        for i in range(1, num_panels + 1):
            scenario = scenarios[i-1] if i <= len(scenarios) else "continued action"

            story_data[f"panel_{i}"] = {
                "scene": f"{prompt} - {scenario}",
                "characters": ["Hero", "Robot"] if "robot" in prompt.lower() else ["Character"],
                "dialogue": f"Panel {i} dialogue" if i % 2 == 0 else ""
            }

        logger.info(f"Generated story structure with {num_panels} panels")
        return story_data

    except Exception as e:
        error_msg = f"Error generating story: {str(e)}"
        logger.error(error_msg)
        traceback.print_exc()

        # Fallback to very simple structure
        fallback_story = {}
        for i in range(1, num_panels + 1):
            fallback_story[f"panel_{i}"] = {
                "scene": prompt,
                "characters": ["Character"],
                "dialogue": f"Panel {i}"
            }
        return fallback_story

# 4. IMAGE GENERATION FUNCTION
def generate_panel_image(panel_data, style="comic book", pipe=None):
    scene = panel_data['scene']
    logger.info(f"Generating image for panel: {scene}")

    try:
        # Construct prompt for image generation - keep it simple
        characters = ", ".join(panel_data["characters"])
        prompt = f"{scene} with {characters}, {style} style"

        # Generate image with minimal steps
        image = pipe(
            prompt,
            num_inference_steps=20,  # Reduced steps
            guidance_scale=7.0,
            height=512,
            width=512,
        ).images[0]

        logger.info("Successfully generated panel image")
        return image

    except Exception as e:
        error_msg = f"Error generating panel image: {str(e)}"
        logger.error(error_msg)
        traceback.print_exc()

        # Return a blank image with error text
        img = Image.new('RGB', (512, 512), color='white')
        d = ImageDraw.Draw(img)
        d.text((10, 10), f"Error: {str(e)}", fill=(0, 0, 0))
        d.text((10, 50), f"Prompt: {scene}", fill=(0, 0, 0))
        return img

# 5. ADD TEXT BUBBLES TO IMAGE
def add_speech_bubble(image, text):
    if not text or text.strip() == "":
        return image  # Skip if no dialogue

    logger.info(f"Adding speech bubble: {text}")

    try:
        # Convert to PIL if needed
        if not isinstance(image, Image.Image):
            image = Image.fromarray(image)

        # Create a copy to draw on
        img_with_text = image.copy()
        draw = ImageDraw.Draw(img_with_text)

        # Simple white box with black text at the top
        width, height = image.size
        margin = 10
        text_width = len(text) * 7  # Approximate width
        text_height = 30

        # Draw white background with black border
        draw.rectangle(
            [(margin, margin),
             (margin + text_width, margin + text_height)],
            fill="white",
            outline="black"
        )

        # Draw text
        draw.text(
            (margin + 5, margin + 5),
            text,
            fill="black"
        )

        return img_with_text

    except Exception as e:
        logger.error(f"Error adding speech bubble: {str(e)}")
        traceback.print_exc()
        return image  # Return original image if error

# 6. ASSEMBLE COMIC PANELS
def create_comic_layout(panel_images, num_cols=2):
    logger.info(f"Creating comic layout with {len(panel_images)} panels")

    try:
        # Handle case with no images
        if not panel_images:
            img = Image.new('RGB', (512, 512), color='white')
            d = ImageDraw.Draw(img)
            d.text((10, 10), "No panels generated", fill=(0, 0, 0))
            return img

        # Determine layout
        num_panels = len(panel_images)
        num_cols = min(num_cols, num_panels)
        num_rows = (num_panels + num_cols - 1) // num_cols

        # Get dimensions from first panel
        panel_width, panel_height = panel_images[0].size

        # Create canvas
        comic_width = panel_width * num_cols
        comic_height = panel_height * num_rows
        comic = Image.new('RGB', (comic_width, comic_height), color='white')

        # Place panels and add borders
        draw = ImageDraw.Draw(comic)
        for i, img in enumerate(panel_images):
            row = i // num_cols
            col = i % num_cols
            x, y = col * panel_width, row * panel_height

            # Paste panel image
            comic.paste(img, (x, y))

            # Draw a border around each panel
            draw.rectangle(
                [(x, y), (x + panel_width - 1, y + panel_height - 1)],
                outline="black",
                width=2
            )

        return comic

    except Exception as e:
        error_msg = f"Error creating comic layout: {str(e)}"
        logger.error(error_msg)
        traceback.print_exc()

        # Return error image
        img = Image.new('RGB', (512, 512), color='white')
        d = ImageDraw.Draw(img)
        d.text((10, 10), f"Error creating layout: {str(e)}", fill=(0, 0, 0))
        return img

# 7. MAIN COMIC GENERATION FUNCTION WITH PROGRESS UPDATES
def generate_comic(prompt, num_panels=4, style="comic book", progress=None):
    logger.info(f"Starting comic generation for prompt: '{prompt}'")

    try:
        # Update progress if available
        if progress:
            progress(0.1, "Initializing models...")

        # Setup models
        tokenizer, text_model = setup_text_model()
        if progress:
            progress(0.2, "Text model loaded")

        image_pipe = setup_image_model()
        if progress:
            progress(0.3, "Image model loaded")

        # Generate story structure
        story_data = generate_story(prompt, num_panels, tokenizer, text_model)
        if progress:
            progress(0.4, "Story generated")

        # Generate panel images
        panel_images = []
        for i, (panel_key, panel_data) in enumerate(story_data.items()):
            if progress:
                progress_val = 0.4 + (0.5 * i / len(story_data))
                progress(progress_val, f"Generating panel {i+1}/{len(story_data)}")

            # Generate image for panel
            panel_image = generate_panel_image(panel_data, style, image_pipe)

            # Add dialogue if present
            if panel_data.get("dialogue"):
                panel_image = add_speech_bubble(panel_image, panel_data["dialogue"])

            panel_images.append(panel_image)

        # Create comic layout
        if progress:
            progress(0.9, "Assembling comic layout")
        comic = create_comic_layout(panel_images)

        if progress:
            progress(1.0, "Comic generated!")

        logger.info("Comic generation complete")
        return comic, story_data

    except Exception as e:
        error_msg = f"Error in comic generation: {str(e)}"
        logger.error(error_msg)
        traceback.print_exc()

        # Create error image
        img = Image.new('RGB', (800, 600), color='white')
        d = ImageDraw.Draw(img)
        d.text((10, 10), "Error generating comic:", fill=(255, 0, 0))
        d.text((10, 40), str(e), fill=(0, 0, 0))
        d.text((10, 70), f"Prompt: {prompt}", fill=(0, 0, 0))
        d.text((10, 100), "Please try again with a simpler prompt or fewer panels", fill=(0, 0, 0))

        # Empty story data
        error_data = {"error": str(e)}

        return img, error_data

# 8. GRADIO UI
def create_ui():
    # Status for progress updates
    status_text = gr.State("")

    def generate_with_progress(prompt, num_panels, style, progress=gr.Progress()):
        def update_progress(progress_val, status):
            progress(progress_val, desc=status)
            return status

        try:
            comic, story_data = generate_comic(
                prompt=prompt,
                num_panels=int(num_panels),
                style=style,
                progress=update_progress
            )
            return comic, json.dumps(story_data, indent=2), "Complete"
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            logger.error(error_msg)
            traceback.print_exc()
            return None, error_msg, "Error"

    # Define interface
    with gr.Blocks() as demo:
        gr.Markdown("# ComicCrafter AI - Generate Comics from Text Prompts")

        with gr.Row():
            with gr.Column():
                prompt_input = gr.Textbox(
                    label="Story Prompt",
                    placeholder="A superhero fights a giant robot in the city",
                    lines=3,
                    value="A superhero with a red cape fights a giant robot in a city"
                )

                with gr.Row():
                    num_panels = gr.Slider(
                        minimum=1,
                        maximum=4,  # Reduced max panels
                        value=2,    # Default to fewer panels
                        step=1,
                        label="Number of Panels"
                    )

                    style = gr.Dropdown(
                        choices=["comic book", "manga", "cartoon", "sketch", "pixel art"],
                        value="comic book",
                        label="Art Style"
                    )

                generate_btn = gr.Button("Generate Comic")
                status = gr.Textbox(label="Status", value="Ready")

            with gr.Column():
                output_image = gr.Image(label="Generated Comic")
                output_story = gr.Textbox(label="Story Structure", lines=10)

        generate_btn.click(
            generate_with_progress,
            inputs=[prompt_input, num_panels, style],
            outputs=[output_image, output_story, status]
        )

        gr.Markdown("""
        ## Tips for Success
        - Start with 2 panels for faster generation
        - Keep prompts simple and focused
        - If you get an error, try a different style or fewer panels
        - The model works best with simple character descriptions
        """)

    return demo

# Run the app
if __name__ == "__main__":
    print("Starting ComicCrafter AI...")
    demo = create_ui()
    demo.launch(debug=True, share=True)